<a href="https://colab.research.google.com/github/sanjaypriyadarsan/Emotion-Detection-from-Uploaded-Images/blob/main/Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# app.py
%%writefile app.py
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np
import mediapipe as mp
from model import EmotionCNN

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionCNN().to(device)
model.load_state_dict(torch.load('emotion_cnn.pth', map_location=device))
model.eval()

# Emotion labels
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Preprocessing for prediction
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

# Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Predict emotion
def predict_emotion(face_img):
    face_img = Image.fromarray(face_img)
    face_tensor = transform(face_img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(face_tensor)
        _, predicted = torch.max(output, 1)
    return EMOTIONS[predicted.item()]

# App UI
st.title("🧠 Emotion Detection from Image")
st.write("Upload an image, and I’ll detect the face and predict the emotion!")

uploaded_file = st.file_uploader("Upload a JPG/PNG image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    img_array = np.array(image)
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    # Face detection
    results = face_detector.process(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))

    if results.detections:
        for det in results.detections:
            bboxC = det.location_data.relative_bounding_box
            h, w, _ = img_rgb.shape
            x, y, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
            face = img_rgb[y:y + h_box, x:x + w_box]

            try:
                emotion = predict_emotion(face)
                st.image(img_rgb, caption="Detected Face", channels="BGR")
                st.success(f"Predicted Emotion: **{emotion.upper()}**")
            except:
                st.warning("Couldn't process face properly. Try another image.")
    else:
        st.warning("No face detected. Please upload a clearer image.")


Writing app.py
